<a href="https://colab.research.google.com/github/sanchianatisha/MidtermTask/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Nama** : Sanchia Natisha Kenzie
## **NPM** : 2106724990
#### **UTS Komputasi Intelegensia**

In [ ]:
# Import library yang diperlukan
!pip install torch torchvision ultralytics opencv-python supervision numpy pandas
!git clone https://github.com/ultralytics/yolov5

import torch
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import time
from pathlib import Path
import supervision as sv
from IPython.display import display, Image as IPImage
from google.colab import files
import os

class DeteksiObjekYOLO:
    def __init__(self):
        """
        Inisialisasi sistem deteksi objek YOLO
        """
        print("Memuat model YOLOv5...")
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

        # Mengatur device ke GPU jika tersedia
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

        # Mengatur model ke mode evaluasi
        self.model.eval()
        print(f"Model dimuat ke {self.device}")

    def proses_frame(self, frame):
        """
        Memproses frame tunggal untuk deteksi objek
        """
        # Periksa apakah frame valid
        if frame is None or frame.size == 0:
            raise ValueError("Frame tidak valid")

        # Konversi BGR ke RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Lakukan inferensi
        hasil = self.model(frame_rgb)

        # Dapatkan deteksi dalam format pandas
        deteksi = hasil.pandas().xyxy[0]

        # Buat salinan frame untuk anotasi
        frame_anotasi = frame.copy()

        # Gambar kotak pembatas dan label
        for idx, deteksi in deteksi.iterrows():
            x1, y1, x2, y2 = map(int, [deteksi['xmin'], deteksi['ymin'],
                                     deteksi['xmax'], deteksi['ymax']])
            label = f"{deteksi['name']} {deteksi['confidence']:.2f}"

            # Gambar kotak pembatas
            cv2.rectangle(frame_anotasi, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Tambahkan label
            cv2.putText(frame_anotasi, label, (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        return deteksi, frame_anotasi

    def proses_video(self, path_video, simpan_output=True):
        """
        Memproses video untuk deteksi objek
        """
        print(f"Memproses video: {path_video}")

        # Periksa keberadaan file
        if not os.path.exists(path_video):
            raise FileNotFoundError(f"File video tidak ditemukan: {path_video}")

        # Buka video
        cap = cv2.VideoCapture(path_video)
        if not cap.isOpened():
            raise ValueError(f"Tidak dapat membuka video: {path_video}")

        # Dapatkan properti video
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"Info Video: {width}x{height} @ {fps}fps")

        # Siapkan video writer
        if simpan_output:
            output_path = 'output_' + os.path.basename(path_video)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_count = 0
        waktu_mulai = time.time()

        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1
                if frame_count % 30 == 0:  # Update progress setiap 30 frame
                    progress = (frame_count / total_frames) * 100
                    print(f"\rProgress: {progress:.1f}% ({frame_count}/{total_frames} frames)", end="")

                # Proses frame
                try:
                    deteksi, frame_anotasi = self.proses_frame(frame)
                except Exception as e:
                    print(f"\nError pada frame {frame_count}: {str(e)}")
                    continue

                # Tampilkan statistik
                fps_current = frame_count / (time.time() - waktu_mulai)
                cv2.putText(frame_anotasi, f'FPS: {fps_current:.2f}', (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Simpan frame yang telah dianotasi
                if simpan_output:
                    out.write(frame_anotasi)

            print(f"\nPemrosesan selesai: {frame_count} frame diproses")
            print(f"FPS rata-rata: {frame_count/(time.time()-waktu_mulai):.2f}")

            if simpan_output:
                print(f"Video output disimpan di: {output_path}")

        except Exception as e:
            print(f"\nTerjadi error: {str(e)}")

        finally:
            # Pembersihan
            cap.release()
            if simpan_output:
                out.release()
            cv2.destroyAllWindows()

def main():
    """
    Fungsi utama untuk menjalankan sistem deteksi objek
    """
    try:
        # Inisialisasi sistem
        print("Menginisialisasi sistem deteksi objek...")
        sistem_deteksi = DeteksiObjekYOLO()

        # Upload video
        print("\nSilakan upload video Anda...")
        uploaded = files.upload()

        if not uploaded:
            print("Tidak ada file yang diupload")
            return

        # Proses video yang diupload
        path_video = next(iter(uploaded))
        print(f"\nMemulai pemrosesan video: {path_video}")
        sistem_deteksi.proses_video(path_video)

    except Exception as e:
        print(f"Terjadi error: {str(e)}")

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 17.6 MB/s eta 0:00:00
Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (11695/11695), done.
Menginisialisasi sistem deteksi objek...
Memuat model YOLOv5...


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 286.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.6s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-10-27 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 220MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Model dimuat ke cuda

Silakan upload video Anda...


...

Progress: 57.9% (150/259 frames)

Progress: 69.5% (180/259 frames)

Progress: 81.1% (210/259 frames)

Progress: 92.7% (240/259 frames)


Pemrosesan selesai: 258 frame diproses
FPS rata-rata: 45.29
Video output disimpan di: output_Video UTS KI.mp4
